In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator, RegressorMixin, clone
from scikeras.wrappers import KerasRegressor
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Conv1D, Flatten, Dense, LSTM
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import joblib
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", message=".*Do not pass an `input_shape`.*")
# -----------------------------
# Load and preprocess the data
# -----------------------------
# Load dataset
df = pd.read_csv(r"C:\ML_work\Data Engineering work\CNN(Model)_weather_prediction\uasin_gishu_weather_data_cleaned.csv", parse_dates=["time"])

# Sort by date
df = df.sort_values(by='time')

# Features and target
features = ['temperature_2m_max', 'temperature_2m_min', 'windspeed_10m_max']
target = 'precipitation_sum'

# Check for missing values and fill them with column mean
df[features + [target]] = df[features + [target]].fillna(df[features + [target]].mean())

# Extract input and output
X = df[features].values
y = df[target].values

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Reshape input for CNN and LSTM: (samples, timesteps, features)
X_scaled_3d = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# -----------------------------
# Define base model functions
# -----------------------------
# CNN Model
def build_cnn_model(filters=64, kernel_size=2, dense_units=32, learning_rate=0.001, input_shape=(1, 3)):
    model = Sequential()
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=input_shape))
    model.add(Flatten())
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dense(1))  # Output layer
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

# LSTM Model
def build_lstm_model(units=50, dense_units=32, learning_rate=0.001, input_shape=(1, 3)):
    model = Sequential()
    model.add(LSTM(units=units, activation='relu', input_shape=input_shape))
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dense(1))  # Output layer
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

# -----------------------------
# Wrap models for scikit-learn
# -----------------------------
cnn_regressor = KerasRegressor(
    model=build_cnn_model,
    model__input_shape=(X_scaled_3d.shape[1], X_scaled_3d.shape[2]),
    verbose=0
)

lstm_regressor = KerasRegressor(
    model=build_lstm_model,
    model__input_shape=(X_scaled_3d.shape[1], X_scaled_3d.shape[2]),
    verbose=0
)

rf_regressor = RandomForestRegressor(random_state=42)
xgb_regressor = XGBRegressor(random_state=42)

# -----------------------------
# Hyperparameter spaces
# -----------------------------
cnn_param_dist = {
    'model__filters': [32, 64, 128],
    'model__kernel_size': [1, 2],
    'model__dense_units': [16, 32, 64],
    'model__learning_rate': [0.001, 0.0001],
    'batch_size': [16, 32],
    'epochs': [30, 50]
}

lstm_param_dist = {
    'model__units': [50, 100],
    'model__dense_units': [16, 32, 64],
    'model__learning_rate': [0.001, 0.0001],
    'batch_size': [16, 32],
    'epochs': [30, 50]
}

rf_param_dist = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5]
}

xgb_param_dist = {
    'n_estimators': [100, 200],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1]
}

# -----------------------------
# Cross-validation and search
# -----------------------------
kfold = KFold(n_splits=3, shuffle=True, random_state=42)
early_stop = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

# Tune CNN
cnn_search = RandomizedSearchCV(
    estimator=cnn_regressor,
    param_distributions=cnn_param_dist,
    cv=kfold,
    n_iter=5,
    scoring='neg_mean_squared_error',
    random_state=42
)
cnn_search.fit(X_scaled_3d, y, callbacks=[early_stop])
print("Best CNN Parameters:", cnn_search.best_params_)

# Tune LSTM
lstm_search = RandomizedSearchCV(
    estimator=lstm_regressor,
    param_distributions=lstm_param_dist,
    cv=kfold,
    n_iter=5,
    scoring='neg_mean_squared_error',
    random_state=42
)
lstm_search.fit(X_scaled_3d, y, callbacks=[early_stop])
print("Best LSTM Parameters:", lstm_search.best_params_)

# Tune Random Forest
rf_search = RandomizedSearchCV(
    estimator=rf_regressor,
    param_distributions=rf_param_dist,
    cv=kfold,
    n_iter=5,
    scoring='neg_mean_squared_error',
    random_state=42
)
rf_search.fit(X_scaled, y)  # RF uses 2D input
print("Best RF Parameters:", rf_search.best_params_)

# Tune XGBoost
xgb_search = RandomizedSearchCV(
    estimator=xgb_regressor,
    param_distributions=xgb_param_dist,
    cv=kfold,
    n_iter=5,
    scoring='neg_mean_squared_error',
    random_state=42
)
xgb_search.fit(X_scaled, y)  # XGBoost uses 2D input
print("Best XGB Parameters:", xgb_search.best_params_)

# -----------------------------
# Create stacking ensemble
# -----------------------------
# Custom wrapper to handle 3D input for neural networks
class StackingWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, model=None, is_3d=False):
        self.model = model
        self.is_3d = is_3d

    def fit(self, X, y):
        if self.model is None:
            raise ValueError("Model must be provided before fitting.")
        if self.is_3d:
            X = X.reshape(X.shape[0], 1, X.shape[1])
        self.model.fit(X, y)
        return self

    def predict(self, X):
        if self.model is None:
            raise ValueError("Model must be provided before predicting.")
        if self.is_3d:
            X = X.reshape(X.shape[0], 1, X.shape[1])
        return self.model.predict(X)

    def get_params(self, deep=True):
        params = {'is_3d': self.is_3d, 'model': self.model}
        if deep and self.model is not None and hasattr(self.model, 'get_params'):
            params.update({f'model__{k}': v for k, v in self.model.get_params(deep=deep).items()})
        return params

    def set_params(self, **params):
        if 'is_3d' in params:
            self.is_3d = params.pop('is_3d')
        if 'model' in params:
            self.model = params.pop('model')
        if self.model is not None and hasattr(self.model, 'set_params'):
            self.model.set_params(**params)
        return self

# Define estimators for stacking
estimators = [
    ('cnn', StackingWrapper(model=clone(cnn_search.best_estimator_), is_3d=True)),
    ('lstm', StackingWrapper(model=clone(lstm_search.best_estimator_), is_3d=True)),
    ('rf', clone(rf_search.best_estimator_)),
    ('xgb', clone(xgb_search.best_estimator_))
]

# Define meta-learner
meta_learner = LinearRegression()

# Create stacking ensemble
stacking_regressor = StackingRegressor(
    estimators=estimators,
    final_estimator=meta_learner,
    cv=kfold
)

# Fit the stacking ensemble
stacking_regressor.fit(X_scaled, y)
print("Stacking ensemble trained successfully.")

# -----------------------------
# Save ensemble model and scaler
# -----------------------------
# Save the entire stacking ensemble as a single entity
joblib.dump(stacking_regressor, "weather_ensemble_model.pkl")
joblib.dump(scaler, "weather_scaler.pkl")

print("Ensemble model and scaler saved successfully.")

Best CNN Parameters: {'model__learning_rate': 0.001, 'model__kernel_size': 1, 'model__filters': 128, 'model__dense_units': 32, 'epochs': 50, 'batch_size': 16}
Best LSTM Parameters: {'model__units': 50, 'model__learning_rate': 0.001, 'model__dense_units': 32, 'epochs': 50, 'batch_size': 32}
Best RF Parameters: {'n_estimators': 100, 'min_samples_split': 5, 'max_depth': None}
Best XGB Parameters: {'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.01}
Stacking ensemble trained successfully.
Ensemble model and scaler saved successfully.
